# Insights from Failed Orders
source: https://platform.stratascratch.com/data-projects/insights-failed-orders

In [2]:
import pandas as pd

In [12]:
offer = pd.read_csv("./datasets/data_offers.csv")
orders = pd.read_csv("./datasets/data_orders.csv")

print(offer.info())
print(orders.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334363 entries, 0 to 334362
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   order_gk  334363 non-null  int64
 1   offer_id  334363 non-null  int64
dtypes: int64(2)
memory usage: 5.1 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10716 entries, 0 to 10715
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_datetime                 10716 non-null  object 
 1   origin_longitude               10716 non-null  float64
 2   origin_latitude                10716 non-null  float64
 3   m_order_eta                    2814 non-null   float64
 4   order_gk                       10716 non-null  int64  
 5   order_status_key               10716 non-null  int64  
 6   is_driver_assigned_key         10716 non-null  int64  
 7   cancellations_time_in_seconds  7307 non-nu

In [19]:
orders.isna().sum()
print(orders.describe())

       origin_longitude  origin_latitude  m_order_eta      order_gk  \
count      10716.000000     10716.000000  2814.000000  1.071600e+04   
mean          -0.964323        51.450541   441.415423  3.000598e+12   
std            0.022818         0.011984   288.006379  2.396261e+07   
min           -1.066957        51.399323    60.000000  3.000550e+12   
25%           -0.974363        51.444643   233.000000  3.000583e+12   
50%           -0.966386        51.451972   368.500000  3.000595e+12   
75%           -0.949605        51.456725   653.000000  3.000623e+12   
max           -0.867088        51.496169  1559.000000  3.000633e+12   

       order_status_key  is_driver_assigned_key  cancellations_time_in_seconds  
count      10716.000000            10716.000000                    7307.000000  
mean           5.590612                0.262598                     157.892021  
std            2.328845                0.440066                     213.366963  
min            4.000000             

['order_datetime']

### 1. Distribution of reasons for failed orders